<a href="https://colab.research.google.com/github/zahra-jazini/data-structures-python/blob/main/%D9%85%D9%82%D8%A7%D9%84%D9%87_%D9%85%D8%AA%D9%86_%DA%A9%D8%A7%D9%88%DB%8C_%D8%AF%D8%A7%D8%B1%D9%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

مرحله 1: پیش‌پردازش نظرات کاربران دارویی
🎯 هدف این مرحله:
تبدیل نظرات کاربران که به‌صورت متن هستن به شکلی که الگوریتم بتونه بفهمه و روی اون‌ها کار کنه. یعنی:

تمیزکاری متن

حذف کلمات بی‌فایده

ریشه‌یابی (lemmatization)

نصب کتابخانه datasets از Hugging Face

In [ ]:
!pip install datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system =

In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
import string
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
from datasets import load_dataset
dataset = load_dataset("Zakia/drugscom_reviews")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/6.72k [00:00<?, ?B/s]

train.tsv:   0%|          | 0.00/84.3M [00:00<?, ?B/s]

test.tsv:   0%|          | 0.00/28.1M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/161297 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/53766 [00:00<?, ? examples/s]

توابع پیش پردازش preprocessing functions

In [ ]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    # حذف اعداد، علائم نگارشی و لینک‌ها
    text = re.sub(r"http\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)
    text = re.sub(r'\d+', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = text.lower()

    # توکن‌سازی و حذف stopwords
    tokens = word_tokenize(text)
    filtered_tokens = [lemmatizer.lemmatize(w) for w in tokens if w not in stop_words and len(w) > 2]

    return ' '.join(filtered_tokens)


بارگذاری دیتاست از Hugging Face

In [ ]:
from datasets import load_dataset

# بارگذاری دیتاست
dataset = load_dataset("Zakia/drugscom_reviews")


 بررسی محتویات دیتاست

In [ ]:
# لیست نام قسمت‌های دیتاست (train و test)
print(dataset)

# نمایش ۵ سطر اول از بخش train
dataset['train'].to_pandas().head()


DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount'],
        num_rows: 161297
    })
    test: Dataset({
        features: ['Unnamed: 0', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount'],
        num_rows: 53766
    })
})


,Unnamed: 0,drugName,condition,review,rating,date,usefulCount
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9.0,"May 20, 2012",27
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8.0,"April 27, 2010",192
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5.0,"December 14, 2009",17
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8.0,"November 3, 2015",10
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9.0,"November 27, 2016",37


تبدیل به DataFrame برای پردازش راحت‌تر

In [ ]:
import pandas as pd

df = dataset['train'].to_pandas()
df.head()


,Unnamed: 0,drugName,condition,review,rating,date,usefulCount
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9.0,"May 20, 2012",27
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8.0,"April 27, 2010",192
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5.0,"December 14, 2009",17
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8.0,"November 3, 2015",10
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9.0,"November 27, 2016",37


بررسی ستون‌های موجود

In [ ]:
print(df.columns)


Index(['Unnamed: 0', 'drugName', 'condition', 'review', 'rating', 'date',
       'usefulCount'],
      dtype='object')


ایجاد ستون برچسب (مثبت / منفی)
ما نیاز داریم که یک ستون جدید درست کنیم به نام sentiment تا مشخص کنیم نظر کاربر مثبته یا منفی. این کار رو با توجه به مقدار rating انجام می‌دیم:

In [ ]:
def label_sentiment(rating):
    if rating >= 8:
        return 'positive'
    elif rating <= 4:
        return 'negative'
    else:
        return 'neutral'

df['sentiment'] = df['rating'].apply(label_sentiment)
df[['rating', 'sentiment']].head(10)


,rating,sentiment
0,9.0,positive
1,8.0,positive
2,5.0,neutral
3,8.0,positive
4,9.0,positive
5,2.0,negative
6,1.0,negative
7,10.0,positive
8,1.0,negative
9,8.0,positive


حذف نظرات خنثی (اختیاری)

In [ ]:
df = df[df['sentiment'] != 'neutral']
df.reset_index(drop=True, inplace=True)
df.shape


(137485, 8)

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

استخراج ویژگی‌ها با Word2Vec
هدف:
تبدیل هر کلمه به یک بردار عددی که معنای اون کلمه رو نشون بده. این باعث می‌شه الگوریتم یادگیری ماشین بتونه معنی کلمات رو بفهمه، نه فقط ظاهر اون‌ها رو.

In [ ]:
!pip install gensim


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 259.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 84.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 15.1 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.15.2
    Uninstalling scipy-1.15.2:
      Successfully uninstalled scipy-1.15.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [ ]:
!pip install --upgrade numpy
!pip install --upgrade gensim


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 91.0 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.3 requires numpy<2.0,>=1.18.5, but you have numpy 2.2.5 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.5 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.2.5 which is incompatible.
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
  Attempting uninstall: numpy
    Found e

In [ ]:
df.columns


Index(['Unnamed: 0', 'drugName', 'condition', 'review', 'rating', 'date',
       'usefulCount', 'sentiment'],
      dtype='object')

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


In [ ]:
!rm -rf /root/nltk_data


In [ ]:
import nltk
nltk.download('punkt', download_dir='/usr/local/nltk_data')
nltk.download('stopwords', download_dir='/usr/local/nltk_data')
nltk.download('wordnet', download_dir='/usr/local/nltk_data')

# افزودن مسیر به مسیرهای جست‌وجوی NLTK
nltk.data.path.append('/usr/local/nltk_data')



In [ ]:
import nltk
import re
import string
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

# اضافه کردن مسیر درست
nltk.data.path.append('/usr/local/nltk_data')
df['clean_review'] = df['review'].astype(str).apply(clean_text)


# حالا با خیال راحت می‌تونی اینا رو فراخوانی کنی
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()


In [ ]:
import re
import string
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

def clean_text_simple(text):
    # حذف لینک‌ها و اعداد
    text = re.sub(r"http\S+|www\S+", '', text)
    text = re.sub(r'\d+', '', text)

    # حذف علائم نگارشی
    text = text.translate(str.maketrans('', '', string.punctuation))

    # حروف کوچک
    text = text.lower()

    # توکن‌سازی ساده و حذف stopwords
    tokens = text.split()
    tokens = [word for word in tokens if word not in ENGLISH_STOP_WORDS and len(word) > 2]

    return ' '.join(tokens)


In [ ]:
df['clean_review'] = df['review'].astype(str).apply(clean_text_simple)
df[['review', 'clean_review']].head()


,review,clean_review
0,"""It has no side effect, I take it in combinati...",effect combination bystolic fish oil
1,"""My son is halfway through his fourth week of ...",son halfway fourth week intuniv concerned bega...
2,"""This is my first time using any form of birth...",time using form birth control glad went patch ...
3,"""Suboxone has completely turned my life around...",suboxone completely turned life feel healthier...
4,"""2nd day on 5mg started to work with rock hard...",day started work rock hard erections experianc...


 آموزش Word2Vec روی متن‌های تمیزشده

In [ ]:
!pip install --upgrade numpy==1.23.5
!pip install --upgrade gensim==4.3.1
import os; os.kill(os.getpid(), 9)  # ریست کردن کرنل


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 66.5 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
blosc2 3.3.2 requires numpy>=1.26, but you have numpy 1.23.5 which is incompatible.
imbalanced-learn 0.13.0 requires numpy<3,>=1.24.3, but you have numpy 1.23.5 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.23.5 which is incompatible.
treescope 0.1.9 requires numpy>=1.25.2, but you have numpy 1.23.5 which is incompatible.
scikit-image 0.25.2 requires numpy>=1.24, but you have numpy 1.23.5 which is incompatible.
bigframes 2.1.0 requires numpy>=1.24.0, but you have numpy 1.23.5 which is incompatible.
xarray 2025.3.1 requires numpy>=1.24, but you have 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.6/26.6 MB 31.7 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset
dataset = load_dataset("Zakia/drugscom_reviews")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
import pandas as pd
df = dataset['train'].to_pandas()


In [ ]:
def label_sentiment(rating):
    if rating >= 8:
        return 'positive'
    elif rating <= 4:
        return 'negative'
    else:
        return 'neutral'

df['sentiment'] = df['rating'].apply(label_sentiment)
df = df[df['sentiment'] != 'neutral']
df.reset_index(drop=True, inplace=True)


In [ ]:
import re
import string
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

def clean_text_simple(text):
    text = re.sub(r"http\S+|www\S+", '', text)
    text = re.sub(r'\d+', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = text.lower()
    tokens = text.split()
    tokens = [word for word in tokens if word not in ENGLISH_STOP_WORDS and len(word) > 2]
    return ' '.join(tokens)

df['clean_review'] = df['review'].astype(str).apply(clean_text_simple)


In [ ]:
sentences = [text.split() for text in df['clean_review']]


In [ ]:
from gensim.models import Word2Vec

# تبدیل متن به لیست کلمات (اگر قبلاً اینو نزدی)
sentences = [text.split() for text in df['clean_review']]

# آموزش مدل Word2Vec
w2v_model = Word2Vec(
    sentences,
    vector_size=100,
    window=5,
    min_count=2,
    workers=4,
    sg=1
)


ساخت بردار ویژگی برای هر نظر (Feature Vectors)

In [11]:
import numpy as np

def get_average_vector(tokens, model, vector_size=100):
    vecs = [model.wv[word] for word in tokens if word in model.wv]
    if len(vecs) == 0:
        return np.zeros(vector_size)
    return np.mean(vecs, axis=0)

# توکن‌سازی
df['tokens'] = df['clean_review'].apply(lambda x: x.split())

# ساخت ویژگی‌ها (X)
X_vectors = np.array([get_average_vector(tokens, w2v_model, 100) for tokens in df['tokens']])


In [14]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y = le.fit_transform(df['sentiment'])  # خروجی: 0 = negative, 1 = positive


In [15]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_vectors, y, test_size=0.2, random_state=42
)


آموزش مدل‌های یادگیری ماشین روی داده‌ها

مدل اول: Logistic Regression (GLM)

In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# ساخت مدل و آموزش
glm_model = LogisticRegression(max_iter=1000)
glm_model.fit(X_train, y_train)

# پیش‌بینی
y_pred = glm_model.predict(X_test)

# ارزیابی
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.8279812343164709

Classification Report:
               precision    recall  f1-score   support

           0       0.76      0.60      0.67      8047
           1       0.85      0.92      0.88     19450

    accuracy                           0.83     27497
   macro avg       0.80      0.76      0.78     27497
weighted avg       0.82      0.83      0.82     27497

